In [6]:
import json
import pandas as pd
import requests

files_path = 'files/'
url="https://toar-data.fz-juelich.de/api/v2/"

variables_url = url + 'variables/'
stations_url = url + 'stationmeta/'
timeseries_url = url + 'timeseries/'
timeseries_data_url = url + 'data/timeseries/'

try:
    with open(files_path + 'toar_apikey.txt', 'r') as f:
        token = f.read().strip()
except FileNotFoundError:
    token = ''
    print(f"API key file not found. Please ensure 'toar_apikey.txt' exists in the {files_path} directory.")

headers = {'AccessToken': token}

# Variables metadata

In [7]:
params = { 'limit': 'None'}
response = requests.get(variables_url, headers=headers, params=params)

In [8]:
if response.status_code == 200:
    variables_json = response.json()
    
    print(f'Correctly loaded metadata from {len(variables_json)} variables:')
else:
    print(f"Error: {response.status_code} - {response.text}")

Correctly loaded metadata from 37 variables:


In [10]:
variables_df = pd.DataFrame(variables_json)
variables_df.rename(columns={'id': 'variable_id'}, inplace=True)
variables_df.head(10)

,name,longname,displayname,cf_standardname,units,chemical_formula,variable_id
0,benzene,benzene,Benzene,mole_fraction_of_benzene_in_air,nmol mol-1,C6H6,1
1,co,carbon monoxide,CO,mole_fraction_of_carbon_monoxide_in_air,nmol mol-1,CO,2
2,no,nitrogen monoxide,NO,mole_fraction_of_nitrogen_monoxide_in_air,nmol mol-1,NO,3
3,pm1,particles up to 1 µm diameter,PM 1,mass_concentration_of_pm1_ambient_aerosol_in_air,µg m-3,,4
4,o3,ozone,Ozone,mole_fraction_of_ozone_in_air,nmol mol-1,O3,5
5,no2,nitrogen dioxide,NO2,mole_fraction_of_nitrogen_dioxide_in_air,nmol mol-1,NO2,6
6,toluene,toluene,Toluene,mole_fraction_of_toluene_in_air,nmol mol-1,C7H8,7
7,so2,Sulphur dioxide,SO2,mole_fraction_of_sulfur_dioxide_in_air,nmol mol-1,SO2,8
8,ethane,Ethane,Ethane,mole_fraction_of_ethane_in_air,nmol mol-1,C2H6,9
9,propane,Propane,Propane,mole_fraction_of_propane_in_air,nmol mol-1,C3H8,10


# Bounding Box
(min_lat, min_lon, max_lat, max_lon in degrees_north/degrees_east) 

In [16]:
min_lat = 19.0487187
max_lat = 19.5927572
min_lon = -99.3649242
max_lon = -98.9403028

# Stations metadata

In [ ]:
params = {
    'limit': 'None',
    'bounding_box': f'{min_lat},{min_lon},{max_lat},{max_lon}'
}

response = requests.get(stations_url, headers=headers, params=params)

In [18]:
if response.status_code == 200:
    stations_json = response.json()
    
    print(f'Correctly loaded metadata from {len(stations_json)} stations:')
else:
    print(f"Error: {response.status_code} - {response.text}")

Correctly loaded metadata from 42 stations:


In [19]:
stations_data = []
for s in stations_json:
    d = {
        'id': s.get('id'),
        'name': s.get('name'),
        'lat': s.get('coordinates', {}).get('lat',-1),
        'lon': s.get('coordinates', {}).get('lng',-1),
        'alt': s.get('coordinates', {}).get('alt',-1),
        'country': s.get('country'),
        'state': s.get('state'),
        'type': s.get('type', s.get('type_of_area'))
    }
    stations_data.append(d)

# optional: convert to DataFrame for easy inspection
stations_df = pd.DataFrame(stations_data)
stations_df.head(5)

,id,name,lat,lon,alt,country,state,type
0,17,Pedregal,19.32515,-99.2041,2326.0,Mexico,Distrito Federal,traffic
1,1318,Santa Fe,19.35730,-99.2628,-999.0,Mexico,,unknown
2,1634,Tlahuac,19.24640,-99.0100,-999.0,Mexico,,unknown
3,1712,Atizapán,19.57700,-99.2542,-999.0,Mexico,,unknown
4,1873,UAM Xochimilco,19.30440,-99.0738,-999.0,Mexico,,unknown


# Time Series

In [ ]:
params = {
    'limit': 'None',
    'station_id': ','.join(stations_df['id'].astype('str').tolist()),
    }

response = requests.get(timeseries_url, headers=headers, params=params)

In [21]:
if response.status_code == 200:
    timeseries_json = response.json()
    
    print(f'Correctly loaded metadata from {len(timeseries_json)} timeseries:')
else:
    print(f"Error: {response.status_code} - {response.text}")

Correctly loaded metadata from 786 timeseries:


In [22]:
timeseries_dict = []

# extract relevant fields
for s in timeseries_json:
    d = {
        'timeseries_id': s.get('id'),
        'variable_id': s.get('variable', {}).get('id'),
        'id': s.get('station', {}).get('id'),
        'sampling_frequency': s.get('sampling_frequency'),
        'start_date': s.get('data_start_date'),
        'end_date': s.get('data_end_date'),
        
    }
    timeseries_dict.append(d)

# convert to DataFrame
timeseries_df = pd.DataFrame(timeseries_dict)
timeseries_df.head(5)

,timeseries_id,variable_id,id,sampling_frequency,start_date,end_date
0,18,5,17,hourly,1986-01-17T07:00:00+00:00,2016-01-01T06:00:00+00:00
1,681,19,1318,irregular data samples of varying length,2016-05-04T22:00:00+00:00,2022-09-29T21:00:00+00:00
2,682,13,1318,irregular data samples of varying length,2016-05-04T22:00:00+00:00,2022-09-29T21:00:00+00:00
3,683,2,1318,irregular data samples of varying length,2016-03-29T12:00:00+00:00,2022-09-29T21:00:00+00:00
4,684,8,1318,irregular data samples of varying length,2016-03-29T08:00:00+00:00,2022-09-29T21:00:00+00:00


In [23]:
# Merge timeseries, stations and variables data
timeseries_df = timeseries_df.merge(variables_df.rename(columns={'name':'variable_name'}),
                    on='variable_id', 
                    how='left')\
                        .merge(stations_df.rename(columns={'name':'station_name'}),
                               on='id', 
                               how='left')

In [ ]:
timeseries_df.groupby(['variable_name']).agg({'start_date': 'min',
                                              'end_date': 'max',
                                              'timeseries_id': 'count'})

,start_date,end_date,timeseries_id
variable_name,,,
cloudcover,2000-01-01T00:00:00+00:00,2022-12-31T23:00:00+00:00,42
co,2016-03-10T08:00:00+00:00,2022-09-29T21:00:00+00:00,37
humidity,2000-01-01T00:00:00+00:00,2022-12-31T23:00:00+00:00,42
irradiance,2006-01-01T07:00:00+00:00,2021-12-31T23:00:00+00:00,42
leaf_area_index,2000-01-01T00:00:00+00:00,2022-12-31T23:00:00+00:00,42
no2,2016-03-10T08:00:00+00:00,2022-09-29T21:00:00+00:00,36
o3,1986-01-17T07:00:00+00:00,2022-09-29T21:00:00+00:00,39
pblheight,2000-01-01T00:00:00+00:00,2022-12-31T23:00:00+00:00,42
pm10,2016-03-10T08:00:00+00:00,2022-10-10T16:00:00+00:00,24


In [ ]:
# filter only data from pollutants
timeseries_df.query('variable_name.isin(["o3","co","no2","so2","pm2p5","pm10"])', inplace=True)
timeseries_df.groupby(['variable_name']).agg({'start_date': 'min', 
                                              'end_date': 'max', 
                                              'timeseries_id': 'count'})

,start_date,end_date,timeseries_id
variable_name,,,
co,2016-03-10T08:00:00+00:00,2022-09-29T21:00:00+00:00,37
no2,2016-03-10T08:00:00+00:00,2022-09-29T21:00:00+00:00,36
o3,1986-01-17T07:00:00+00:00,2022-09-29T21:00:00+00:00,39
pm10,2016-03-10T08:00:00+00:00,2022-10-10T16:00:00+00:00,24
pm2p5,2016-03-10T08:00:00+00:00,2022-10-04T13:00:00+00:00,26
so2,2016-03-10T08:00:00+00:00,2022-09-29T21:00:00+00:00,36


In [26]:
timeseries_df.to_csv(files_path + 'timeseries_metadata.csv', index=False)

# Time Series Data

In [ ]:
import time

params = {
    'limit': 'None',
    'format': 'csv',
    }

for _, row in timeseries_df.iterrows():
    
    # request the data from TOAR  API
    ts_id = str(row['timeseries_id'])
    response = requests.get(timeseries_data_url + ts_id, 
                            headers=headers,
                            params=params)
    

    if response.status_code == 200:
        
        # decode bytes to string (utf-8, replace invalid bytes)
        text = response.content.decode('utf-8', errors='replace') if isinstance(response.content, (bytes, bytearray)) else str(response.content)
        metadata = [line[1::] for line in text.splitlines() if line.startswith('#')]
        metadata = json.loads('\n'.join(metadata))
        data = [line for line in text.splitlines() if not line.startswith('#')]
        data = '\n'.join(data)
        
        # save CSV with the timeseries id in the filename
        filename = files_path + f"timeseries_{metadata['id']}.csv"
        
        with open(filename, "wb") as fh:
            fh.write(data.encode('utf-8'))
        print(f"Saved CSV to {filename} ({len(response.content)} bytes)")
        time.sleep(5)
    else:
        print(f"Failed to get CSV: {response.status_code} - {getattr(response, 'text', '')}")
    

Saved CSV to files/timeseries_18.csv (15544425 bytes)
Saved CSV to files/timeseries_681.csv (1212011 bytes)
Saved CSV to files/timeseries_682.csv (1225885 bytes)
Saved CSV to files/timeseries_683.csv (1493190 bytes)
Saved CSV to files/timeseries_684.csv (1686684 bytes)
Saved CSV to files/timeseries_685.csv (1674279 bytes)
Saved CSV to files/timeseries_686.csv (1588123 bytes)
Saved CSV to files/timeseries_1281.csv (1553744 bytes)
Saved CSV to files/timeseries_1282.csv (1487084 bytes)
Saved CSV to files/timeseries_1283.csv (1454930 bytes)
Saved CSV to files/timeseries_1284.csv (1714804 bytes)
Saved CSV to files/timeseries_1285.csv (1362472 bytes)
Saved CSV to files/timeseries_1448.csv (1485562 bytes)
Saved CSV to files/timeseries_1449.csv (1788423 bytes)
Saved CSV to files/timeseries_1450.csv (1404468 bytes)
Saved CSV to files/timeseries_1451.csv (1747606 bytes)
Saved CSV to files/timeseries_1452.csv (1722527 bytes)
Saved CSV to files/timeseries_1853.csv (1707813 bytes)
Saved CSV to file

# Build database

In [21]:
df = pd.DataFrame()
for _, row in timeseries_df.iterrows():
    ts_id = str(row['timeseries_id'])
    station = str(row['station_name'])
    variable = str(row['variable_name'])
    ts_df = pd.read_csv(files_path + f"timeseries_{ts_id}.csv",parse_dates=['datetime'])
    ts_df = ts_df[['datetime','value']].assign(station=station, variable=variable, inplace=True)
    
    df = pd.concat([df, ts_df], ignore_index=True)

In [26]:
df = df.groupby(['station','variable','datetime']).agg({'value':list}).reset_index()
df['value'] = df['value'].apply(lambda x: x[0] if len(x) == 1 else max(x))

In [28]:
df.to_csv(files_path + 'timeseries_data.csv', index=False)